# Covid Tweet Classification

In [57]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim

In [37]:
train_data = pd.read_csv("Corona_NLP_train.csv",encoding= 'ISO-8859-1')
test_data = pd.read_csv("Corona_NLP_test.csv",encoding= 'ISO-8859-1')

print('Train data shape',train_data.shape)
print('Test data shape',test_data.shape)

Train data shape (41157, 6)
Test data shape (3798, 6)


In [38]:
train_data.isnull().sum()

UserName            0
ScreenName          0
Location         8590
TweetAt             0
OriginalTweet       0
Sentiment           0
dtype: int64

In [39]:
test_data.isnull().sum()

UserName           0
ScreenName         0
Location         834
TweetAt            0
OriginalTweet      0
Sentiment          0
dtype: int64

In [40]:
train_data = train_data[['OriginalTweet','Sentiment']]
test_data = test_data[['OriginalTweet','Sentiment']]

In [41]:
train_data['Sentiment'].nunique()

5

There are 5 unique types of tweets.

In [42]:
label_encoder = LabelEncoder()
train_data['Sentiment'] = label_encoder.fit_transform(train_data['Sentiment'])
test_data['Sentiment'] = label_encoder.transform(test_data['Sentiment'])

Extremely Negative : 0, Extremely Positive : 1, Negative : 2, Neutral : 3, Positive : 4

In [62]:
# Check if a GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load and preprocess your data
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_encodings = tokenizer(list(train_data['OriginalTweet']), truncation=True, padding=True, max_length=128, return_tensors='pt')
test_encodings = tokenizer(list(test_data['OriginalTweet']), truncation=True, padding=True, max_length=128, return_tensors='pt')

train_labels = torch.tensor(list(train_data['Sentiment']))
test_labels = torch.tensor(list(test_data['Sentiment']))

# Create data loaders
batch_size = 8
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], train_labels)
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], test_labels)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Load and configure the model for training
num_classes = 5
model_name = 'bert-base-uncased'

model = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_classes)
model.to(device)  # Move the model to the GPU if available

# Define optimizer and loss
optimizer = optim.AdamW(model.parameters(), lr=2e-5)
criterion = torch.nn.CrossEntropyLoss()

# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Evaluate the model on the test set
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for batch in test_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)  # Move data to GPU
        outputs = model(input_ids, attention_mask=attention_mask)
        _, predicted = torch.max(outputs.logits, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Accuracy on the test set: {accuracy * 100:.2f}%')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RuntimeError: CUDA out of memory. Tried to allocate 90.00 MiB (GPU 0; 4.00 GiB total capacity; 3.37 GiB already allocated; 0 bytes free; 3.41 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF